In [1]:
import sys
sys.path.insert(0, "./src")

import numpy as np
import torch
from tqdm import tqdm
from src.LSTM.regressor import Regressor
from src.LSTM.LSTM import LSTMcell
from src.LSTM.utils import mse_loss, mse_loss_d, sigmoid_d, tanh_d

In [2]:
X = torch.randint(0, 10, (16000, 1, 1), dtype=torch.float32)
y = X.sum(dim=1)
x = X[:16]
y_gt = y[:16]

In [3]:
# Initiazte here just to reset the model states
cell = LSTMcell(1, 1)
regressor = Regressor(1, 1)

In [ ]:
losses = []

for i in range(100000):
    hidden_states = cell.forward(x)
    # Obtain the last hidden states to be fed to the regressor
    last_hidden_states = hidden_states[:, -1, :]
    regressed_values = regressor.forward(last_hidden_states)

    # Calculate losses
    loss = mse_loss(y_gt, regressed_values)
    loss_d = mse_loss_d(y_gt, regressed_values)

    # Backward
    chained_d = regressor.backward(loss_d, 0.01)
    c_state_d, h_state_d, input_d = cell.backward(chained_d, 0.01)
    
    if i % 10000 == 9999:
        print(sum(losses)/len(losses))
        losses.clear()
    else:
        losses.append(loss)

tensor(0.0868)
tensor(0.0861)
tensor(0.0857)
tensor(0.0854)
tensor(0.0852)
tensor(0.0851)


In [8]:
x

tensor([[[6.]],

        [[8.]],

        [[4.]],

        [[3.]],

        [[3.]],

        [[4.]],

        [[5.]],

        [[5.]],

        [[1.]],

        [[8.]],

        [[6.]],

        [[7.]],

        [[8.]],

        [[7.]],

        [[6.]],

        [[2.]]])

In [13]:
out = cell.forward(x, use_grads=False)
out_y = regressor.forward(out, require_grads=False)

In [28]:
out_y.squeeze(1)

tensor([[6.2165],
        [7.6897],
        [3.6165],
        [2.6570],
        [2.6570],
        [3.6165],
        [4.9268],
        [4.9268],
        [1.6384],
        [7.6897],
        [6.2165],
        [7.1431],
        [7.6897],
        [7.1431],
        [6.2165],
        [2.0212]])

In [29]:
x.squeeze(1)

tensor([[6.],
        [8.],
        [4.],
        [3.],
        [3.],
        [4.],
        [5.],
        [5.],
        [1.],
        [8.],
        [6.],
        [7.],
        [8.],
        [7.],
        [6.],
        [2.]])